In [1]:
#Daily Payments Processed for Vendors - in CARS Financial Folder
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta
import openpyxl
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Alignment, PatternFill
import os

# Paths
base_path = Path(r"Z:\work\ach_eobr\DailyArchive")
mapping_path = Path(r"Z:\work\ach_eobr\DataSource\VendorList.csv")
log_file_path = base_path / 'error_log.txt'

# Load mapping data
mapping_data = pd.read_csv(mapping_path)
filtered_data = pd.read_excel('Daily Payments Processed for Vendors (140).xlsx', skiprows=3, header=0)
filtered_data = filtered_data.drop('Unnamed: 0', axis=1)
filtered_data["Check #"] = filtered_data["Check #"].astype(str)
filtered_data = filtered_data[~filtered_data["Check #"].str.startswith("0")]
data_copy = filtered_data

# Function to get the previous business day
def get_previous_business_day():
    result = datetime.now()
    if result.weekday() == 0:  # Monday
        result = result - timedelta(days=3)
    else:
        result = result - timedelta(days=1)
    return result

# Prepare date information
previous_business_day = get_previous_business_day()
formatted_previous_day = previous_business_day.strftime('%Y-%m-%d')

start_date = ('2025-01-30')
end_date = ('2025-01-30')

datestamp = ('20250130')
date_folder = base_path / datestamp

#Filter out data that is not from previous business day.
filtered_data = filtered_data[(filtered_data['Transaction Date'] >= start_date) & (filtered_data['Transaction Date'] <= end_date)]


#FORMATTING
filtered_data['Service Start'] = pd.to_datetime(filtered_data['Service Start'])
filtered_data['Service Start'] = filtered_data['Service Start'].dt.strftime('%Y-%m-%d')
filtered_data['Service End'] = pd.to_datetime(filtered_data['Service End'])
filtered_data['Service End'] = filtered_data['Service End'].dt.strftime('%Y-%m-%d')
filtered_data['Service End'] = pd.to_datetime(filtered_data['Service End'])
filtered_data['Service End'] = filtered_data['Service End'].dt.strftime('%Y-%m-%d')
filtered_data['Transaction Date'] = pd.to_datetime(filtered_data['Transaction Date'])
filtered_data['Transaction Date'] = filtered_data['Transaction Date'].dt.strftime('%Y-%m-%d')
filtered_data['Payee Tax ID'] = filtered_data['Payee Tax ID'].astype(str)
mapping_data['VendorTaxID'] = mapping_data['VendorTaxID'].astype(str)
filtered_data['Payee Tax ID'] = filtered_data['Payee Tax ID'].str.split('.').str[0]
mapping_data['VendorTaxID'] = mapping_data['VendorTaxID'].str.split('.').str[0]

# Enrich filtered_data with VendorCode from mapping_data
filtered_data = filtered_data.merge(
    mapping_data[['VendorTaxID', 'VendorCode']],
    how='left',
    left_on='Payee Tax ID',
    right_on='VendorTaxID'
)

#Dropping Unneeded
filtered_data['VendorCode'].fillna('OTHER_VENDOR_PAYMENTS', inplace=True)
filtered_data.drop(columns=['Payee Tax ID'], inplace=True)
filtered_data.drop(columns=['Check #'], inplace=True)

# Ensure date_folder exists
if not date_folder.exists():
    raise Exception(f"Date folder {date_folder} does not exist.")

# Function to sanitize file names
def sanitize_filename(name):
    return name.replace('/', '_').replace('\\', '_').replace(':', '_').replace('*', '_')\
        .replace('?', '_').replace('"', '_').replace('<', '_').replace('>', '_').replace('|', '_')

# Open log file
with open(log_file_path, 'w') as log_file:
    for payee, group in filtered_data.groupby('Payee'):
        try:
            vendor_code = group['VendorCode'].iloc[0]
            
            # Determine target folder
            if vendor_code == "OTHER_VENDOR_PAYMENTS":
                target_folder = date_folder / "OTHER_VENDOR_PAYMENTS"
                # Create OTHER_VENDOR_PAYMENTS folder only if needed
                if not target_folder.exists():
                    target_folder.mkdir(parents=True, exist_ok=True)
            else:
                target_folder = date_folder / vendor_code

            # Ensure the target folder exists
            if not target_folder.exists():
                raise Exception(f"Folder for vendor '{vendor_code}' does not exist.")

            sanitized_payee = sanitize_filename(payee)
            file_path = target_folder / f"{sanitized_payee}_transactions_{datestamp}.xlsx"

            # Export to Excel
            group.to_excel(file_path, index=False)

            # Format Excel file
            wb = load_workbook(file_path)
            ws = wb.active

            # Style headers
            header_font = Font(bold=True, color="FFFFFF")
            header_fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")
            header_alignment = Alignment(horizontal="center", vertical="center")

            for cell in ws[1]:
                cell.font = header_font
                cell.fill = header_fill
                cell.alignment = header_alignment

            # Adjust column widths
            for col in ws.columns:
                max_length = 0
                col_letter = get_column_letter(col[0].column)
                for cell in col:
                    if cell.value is not None:
                        max_length = max(max_length, len(str(cell.value)))
                ws.column_dimensions[col_letter].width = max_length + 2

            # Add total row
            last_row = ws.max_row + 1
            ws.cell(row=last_row, column=8).value = 'Total'
            ws.cell(row=last_row, column=9).value = group['Amount'].sum()

            # Save workbook
            wb.save(file_path)

        except Exception as e:
            # Log errors
            log_file.write(f"Error processing payee '{payee}': {str(e)}\n")
            print(f"Error processing payee '{payee}': {str(e)}")

print("Process completed. Check the error log for details.")


C:\Users\gduln001\AppData\Local\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\gduln001\AppData\Local\Temp\1\ipykernel_35080\3451641956.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Service Start'] = pd.to_datetime(filtered_data['Service Start'])
C:\Users\gduln001\AppData\Local\Temp\1\ipykernel_35080\3451641956.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

Process completed. Check the error log for details.


In [2]:
filtered_data

,Payee,Transaction Date,Claimant Name,Claim Number,Invoice Number,Bill Review Bill ID,Service Start,Service End,Amount,VendorTaxID,VendorCode
0,LTD AMERICA TOTAL CARE,2025-01-30,"Calvino Gonzalez, Mayelin",WDWW2023234223,IP111524223821,99997-X-13255940-0,2024-10-30,2024-11-15,1250.00,203003799,LTD
1,LTD AMERICA TOTAL CARE,2025-01-30,"Clerger, Marie R",WDWW2022222114,TP12524225620,99997-X-13262639-0,2024-11-19,2024-12-05,1049.40,203003799,LTD
2,LTD AMERICA TOTAL CARE,2025-01-30,"Diaz, Araceli",DLRW2024103842,CC124249427,99997-X-13262507-0,2024-12-04,2024-12-04,10.15,203003799,LTD
3,LTD AMERICA TOTAL CARE,2025-01-30,"Flores Ramirez, Adriana",DLRW2024102205,CC63249391,99997-X-13262528-0,2024-06-03,2024-06-03,15.95,203003799,LTD
4,LTD AMERICA TOTAL CARE,2025-01-30,"Flores Ramirez, Adriana",DLRW2024102205,CC819249413,99997-X-13262521-0,2024-08-19,2024-08-19,13.05,203003799,LTD
...,...,...,...,...,...,...,...,...,...,...,...
90,KIMBERLY J EVANS OTR/L CHT PC,2025-01-30,"Salazar, Rebecca",DLRW2024102702,102227,99997-H-13262036-0,2025-01-14,2025-01-14,157.73,821348823,KIMBERLY_EVANS
91,KIMBERLY J EVANS OTR/L CHT PC,2025-01-30,"Salazar, Rebecca",DLRW2024102702,102227,99997-H-13264758-0,2025-01-16,2025-01-16,166.30,821348823,KIMBERLY_EVANS
92,DOCTORS HOSPITAL BAHAMAS LTD,2025-01-30,"Camargo Vega, Joseph J",DCLW2025305114,VAA016717590,99997-U-13262125-0,2025-01-12,2025-01-12,3466.94,980160433,DOCTORS_HOSPITAL
93,CENTRO MEDICO DE COZUMEL S.A. DE C.,2025-01-30,"Johnson, Kadian A",DCLW2024004038,104281,99997-Y-13257822-0,2024-12-02,2024-12-02,607.28,NaN,OTHER_VENDOR_PAYMENTS
